<a target="_blank" href="https://shiyanlou.com/louplus/ml"><img style="float: right;" src="https://doc.shiyanlou.com/document-uid214893labid7506timestamp1555400601684.png"></a>

## 蘑菇分类模型部署和推理

---

<i class="fa fa-exclamation-circle" aria-hidden="true"> 以下内容仅包含挑战需学员补充部分的参考答案，完整挑战请到原课程页面查看。</i>

---

你可以参考以下内容，或直接启动 [Kaggle Kernels](https://www.kaggle.com/louplus/mushroom) 版本。

### 加载数据集

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("https://labfile.oss.aliyuncs.com/courses/1321/mushrooms.csv")
df.tail()

### 模型训练和保存

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib

X = pd.get_dummies(df.iloc[:, 1:])  # 读取特征并独热编码
y = df['class']  # 目标值

model = RandomForestClassifier()  # 随机森林
print(cross_val_score(model, X, y, cv=5).mean())  # 交叉验证结果

model.fit(X, y)  # 训练模型
joblib.dump(model, "mushrooms.pkl")  # 保存模型
print("model saved.")

### 构建 Flask Web

In [ ]:
%%writefile predict.py
# 将此单元格代码写入 predict.py 文件方便后面执行
import joblib
import pandas as pd
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/", methods=["POST"])  # 请求方法为 POST
def inference():
    query_df = pd.DataFrame(request.json)  # 将 JSON 变为 DataFrame
    
    df = pd.read_csv("https://labfile.oss.aliyuncs.com/courses/1321/mushrooms.csv")  # 读取数据
    X = pd.get_dummies(df.iloc[:, 1:])  # 读取特征并独热编码
    query = pd.get_dummies(query_df).reindex(columns=X.columns, fill_value=0)  # 将请求数据 DataFrame 处理成独热编码样式
    
    clf = joblib.load('mushrooms.pkl')  # 加载模型
    prediction = clf.predict(query)  # 模型推理
    return jsonify({"prediction": list(prediction)})  # 返回推理结果

### Notebook 中必须以子进程才能正常启动 Flask

In [ ]:
import time
import subprocess as sp

# 启动子进程执行 Flask app
server = sp.Popen("FLASK_APP=predict.py flask run", shell=True)
time.sleep(5)  # 等待 5 秒保证 Flask 启动成功
server

In [ ]:
import json

# 从测试数据中取 1 条用于测试推理
df_test = pd.read_csv("https://labfile.oss.aliyuncs.com/courses/1321/mushrooms_test.csv")
sample_data = df.sample(1).to_json(orient='records')
sample_json = json.loads(sample_data)
sample_json

In [ ]:
import requests

requests.post(url="http://localhost:5000", json=sample_json).content  # 建立 POST 请求，并发送数据请求

In [ ]:
server.terminate()  # 结束子进程，关闭端口占用

---

<img src="https://doc.shiyanlou.com/document-uid214893labid7506timestamp1545810029884.png">